In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency, spearmanr, skew, pearsonr
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import confusion_matrix

#SMOTE
from imblearn.over_sampling import SMOTE

#Mô hình XG Boost
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

#Random Forest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
!pip install catboost
from catboost import CatBoostClassifier

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/ieee-fraud-detection/changed datasets/ieee-fraud-detection-changed.csv')
data

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0,86506,50.000,H,4497.0,514.0,150.0,mastercard,102.0,...,144.0,New,NotFound,samsung browser 6.2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,0,86535,15.000,H,2803.0,100.0,150.0,visa,226.0,...,500.0,New,NotFound,mobile safari 11.0,T,F,F,T,mobile,iOS Device
2,2987010,0,86549,75.887,C,16496.0,352.0,117.0,mastercard,134.0,...,142.0,Found,Found,chrome 62.0,F,F,T,T,desktop,Windows
3,2987011,0,86555,16.495,C,4461.0,375.0,185.0,mastercard,224.0,...,507.0,New,NotFound,chrome 62.0,F,F,T,T,desktop,Windows
4,2987016,0,86620,30.000,H,1790.0,555.0,150.0,visa,226.0,...,575.0,Found,Found,chrome 62.0,T,F,T,T,desktop,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,0,15810802,48.877,C,12019.0,305.0,106.0,mastercard,224.0,...,139.0,Found,Found,chrome 66.0 for android,F,F,T,F,mobile,F3111 Build/33.3.A.1.97
144229,3577526,1,15810876,250.000,R,1214.0,174.0,150.0,visa,226.0,...,333.0,New,NotFound,chrome 55.0 for android,T,F,T,F,mobile,A574BL Build/NMF26F
144230,3577529,0,15810912,73.838,C,5096.0,555.0,185.0,mastercard,137.0,...,411.0,New,NotFound,chrome 65.0 for android,F,F,T,F,mobile,Moto E (4) Plus Build/NMA26.42-152
144231,3577531,0,15810935,400.000,R,6019.0,583.0,150.0,visa,226.0,...,368.0,Found,Found,chrome 66.0,T,F,T,F,desktop,MacOS


In [ ]:
#Xóa đi các columns vô nghĩa (chỉ có 1 giá trị, hoặc là Id):
data = data.drop(columns = ['C5', 'C9', 'V107', 'V305', 'TransactionID'])

In [ ]:
def detect_cols_types(data):
  category_cols = []
  numeric_cols = []
  for i in data.columns:
    if data[i].dtype == 'object':
      category_cols.append(i)
    elif len(data[i].unique()) < 15:
      category_cols.append(i)
    else:
      numeric_cols.append(i)
  return category_cols, numeric_cols

category_cols, numeric_cols = detect_cols_types(data)

# **FEATURES SELECTION**

In [ ]:
for i in category_cols:
  print(data[i].value_counts())
  print('\n')

isFraud
0    132915
1     11318
Name: count, dtype: int64


ProductCD
C    62192
R    37548
H    32908
S    11585
Name: count, dtype: int64


card4
visa                89483
mastercard          44186
american express     8298
discover             2266
Name: count, dtype: int64


card6
credit         75268
debit          68950
charge card       15
Name: count, dtype: int64


P_emaildomain
gmail.com           67531
hotmail.com         25782
anonymous.com       17840
yahoo.com           11667
aol.com              4306
outlook.com          2346
comcast.net          2196
yahoo.com.mx         1182
msn.com              1069
icloud.com           1022
verizon.net           758
live.com              737
sbcglobal.net         724
live.com.mx           653
me.com                633
att.net               577
cox.net               515
bellsouth.net         501
outlook.es            396
mac.com               304
hotmail.fr            290
hotmail.es            270
optonline.net         265
ymail.com  

***Kiểm định Chi-squared cho các columns Phân loại***

In [ ]:
remove_obj_cols =[]

#Chi-square test cho các cột Object so với cột Output của data
for i in category_cols:
  if i == 'isFraud':
    continue
  else:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(data[i], data['isFraud']))
    print(i, '---', pval)
    if pval > 0.05:
      remove_obj_cols.append(i)
    else:
      continue
print(remove_obj_cols)

ProductCD --- 0.0
card4 --- 5.254067623833289e-75
card6 --- 3.725583010962303e-54
P_emaildomain --- 0.0
R_emaildomain --- 0.0
V98 --- 3.9914631424156165e-10
V108 --- 0.0
V109 --- 0.0
V110 --- 0.0
V111 --- 0.0
V112 --- 0.0
V113 --- 0.0
V114 --- 0.0
V115 --- 0.0
V116 --- 0.0
V117 --- 5.963837348660794e-252
V118 --- 2.364317221383829e-235
V119 --- 2.5115030385936443e-244
V120 --- 1.079389047843313e-86
V121 --- 5.640892964531436e-72
V122 --- 4.300620116416036e-81
V173 --- 2.2903483603363654e-12
V174 --- 1.9040944747420618e-12
V194 --- 0.0
V240 --- 0.005629783661409558
V241 --- 0.2718676378701167
V260 --- 0.0
V284 --- 6.1241228994725274e-18
V286 --- 1.601299214225599e-28
V288 --- 5.140651641511074e-13
V289 --- 6.665788753119167e-37
V297 --- 1.9257902192026077e-20
V300 --- 0.2546702657572877
V301 --- 0.05890682174860353
id_12 --- 9.994884898224519e-28
id_15 --- 1.7838078747849106e-279
id_16 --- 2.593865578442457e-309
id_28 --- 3.561015446940426e-244
id_29 --- 1.3634362604840606e-271
id_31 --

In [ ]:
data = data.drop(columns = remove_obj_cols)

***Spearman Correlation cho các columns Số***

In [ ]:
spearman_cols = numeric_cols + ['isFraud']
spearman_df = data[spearman_cols]
correlation_results = []

# Duyệt qua tất cả các cột trong DataFrame
for col in spearman_df.columns:
  if col == 'isFraud':
    continue
  else:
    if spearman_df[col].nunique() > 1:
      corr, p_value = spearmanr(spearman_df[col], spearman_df['isFraud'])
      correlation_results.append((col, corr, p_value))
    else:
        print(f"Cột {col} có giá trị không thay đổi và bị bỏ qua.")

# Chuyển kết quả thành DataFrame để dễ quan sát
correlation_df = pd.DataFrame(correlation_results, columns=['Feature', 'Spearman Correlation', 'p-value'])

In [ ]:
correlation_df = correlation_df.sort_values(by='Spearman Correlation', ascending=False)
correlation_df

,Feature,Spearman Correlation,p-value
6,C1,0.340040,0.000000e+00
131,V258,0.311321,0.000000e+00
74,V199,0.303481,0.000000e+00
130,V257,0.301217,0.000000e+00
66,V190,0.298398,0.000000e+00
...,...,...,...
189,id_06,-0.048908,4.220486e-77
17,C14,-0.051644,9.211209e-86
5,card5,-0.058515,1.359495e-109
3,card2,-0.060937,1.054211e-118


In [ ]:
#Lọc ra các Feature có giá trị Corr > 0.15
delected_spearman_df = correlation_df[correlation_df['Spearman Correlation'] < 0.25]
delected_cols = delected_spearman_df['Feature'].to_list()
len(delected_cols)

178

In [ ]:
data = data.drop(columns = delected_cols)

# ***Encoding***

In [ ]:
obj_cols = data.select_dtypes(include='object').columns.tolist()
for i in obj_cols:
  print(data[i].value_counts())
  print('\n')

ProductCD
C    62192
R    37548
H    32908
S    11585
Name: count, dtype: int64


card4
visa                89483
mastercard          44186
american express     8298
discover             2266
Name: count, dtype: int64


card6
credit         75268
debit          68950
charge card       15
Name: count, dtype: int64


P_emaildomain
gmail.com           67531
hotmail.com         25782
anonymous.com       17840
yahoo.com           11667
aol.com              4306
outlook.com          2346
comcast.net          2196
yahoo.com.mx         1182
msn.com              1069
icloud.com           1022
verizon.net           758
live.com              737
sbcglobal.net         724
live.com.mx           653
me.com                633
att.net               577
cox.net               515
bellsouth.net         501
outlook.es            396
mac.com               304
hotmail.fr            290
hotmail.es            270
optonline.net         265
ymail.com             264
web.de                237
charter.net        

***Label encoding tự động cho các columns:***

In [ ]:
for i in ['ProductCD', 'card4', 'card6', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'id_31']:
  value_counts = data[i].value_counts().index
  value_to_label = {value: idx for idx, value in enumerate(value_counts)}
  data[i] = data[i].map(value_to_label)

***Encoding cho 2 columns là P_emaildomain và R_emaildomain, 2 cột này liên quan đến nhau***

In [ ]:
def email_domain_mapping(data, columns_names=['P_emaildomain', 'R_emaildomain']):
    # Tính value_counts() cho cả hai cột và cộng lại
    P_counts = data['P_emaildomain'].value_counts()
    R_counts = data['R_emaildomain'].value_counts()

    total_counts = P_counts.add(R_counts, fill_value=0) # Tính tổng số lần xuất hiện của mỗi domain

    # Sắp xếp domains theo số lần xuất hiện giảm dần
    sorted_domains = total_counts.sort_values(ascending=False).index.tolist()

    # Gán nhãn mã hóa, với domain xuất hiện nhiều hơn được ưu tiên mã hóa trước
    domain_mapping = {domain: idx for idx, domain in enumerate(sorted_domains)}

    # Áp dụng mã hóa cho từng cột trong columns_names
    for col in columns_names:
        data[col] = data[col].map(domain_mapping)

    return data
data = email_domain_mapping(data)

***Corr Matrix***

In [ ]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,isFraud,ProductCD,card4,card6,P_emaildomain,R_emaildomain,C1,C2,C11,V98,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V173,V174,V186,V187,V190,V194,V199,V200,V201,V232,V233,V240,V242,V243,V244,V246,V257,V258,V260,V284,V286,V288,V289,V297,id_12,id_15,id_16,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
isFraud,1.000000,-0.110005,-0.014244,-0.041268,-0.060268,-0.048920,0.034436,0.040319,0.029325,-0.014529,0.174387,0.172967,0.173753,0.173174,0.171955,0.172591,0.175591,0.169311,0.172603,0.089087,0.086163,0.087764,0.052481,0.047359,0.050594,-0.020613,0.016857,0.147014,0.023920,0.202539,0.144997,0.228086,0.315150,0.324268,0.051848,0.043872,0.002627,0.342652,0.203143,0.345998,0.348363,0.363684,0.285647,0.118512,-0.024650,-0.031103,-0.010912,0.001009,-0.028289,-0.028771,-0.060399,-0.099017,-0.087896,-0.092740,0.061410,0.144881,-0.036065,-0.023727,-0.065621,0.068353,0.025545
ProductCD,-0.110005,1.000000,0.050200,-0.126921,-0.011988,-0.004433,-0.076772,-0.085417,-0.076203,0.163627,-0.058965,-0.052771,-0.057101,-0.064883,-0.061101,-0.062898,-0.064615,-0.042299,-0.056136,-0.031067,-0.024524,-0.028341,-0.004905,0.009332,0.001794,-0.105509,-0.029460,-0.037123,-0.003711,-0.062938,-0.034789,-0.057238,-0.093984,-0.091947,0.057682,0.060708,-0.017185,-0.100419,-0.059606,-0.109238,-0.118554,-0.106306,-0.081802,0.089315,-0.029325,-0.118103,-0.070978,-0.036494,0.201241,0.427536,-0.177470,0.021182,-0.130234,-0.125520,-0.097725,-0.816194,0.124258,0.022392,0.541192,-0.138105,-0.127374
card4,-0.014244,0.050200,1.000000,-0.214732,0.012631,0.033378,-0.005254,-0.005729,-0.004614,0.042756,-0.011810,-0.011470,-0.011666,-0.013821,-0.012905,-0.013395,-0.012186,-0.006086,-0.009944,-0.004326,0.000559,-0.002960,-0.001299,0.004403,0.001119,-0.000253,0.010144,0.002638,0.009378,-0.001215,-0.018166,0.001469,-0.013749,-0.010200,0.032018,0.031037,0.000889,-0.011310,0.002375,-0.012580,-0.013745,-0.009220,0.000598,0.012425,0.016293,-0.004693,0.011997,0.013619,0.083117,0.079351,-0.031991,-0.019201,-0.035797,-0.035221,-0.018177,-0.083713,-0.006272,-0.004817,-0.021974,-0.051234,-0.027095
card6,-0.041268,-0.126921,-0.214732,1.000000,-0.060183,-0.050792,0.013334,0.015822,0.012349,-0.038355,0.003668,0.002746,0.003341,0.006320,0.005212,0.005740,0.000545,-0.001175,0.000125,0.000177,0.000351,0.000701,-0.001724,-0.004644,-0.003364,0.012080,0.010614,-0.029331,-0.032241,-0.028392,-0.021341,-0.035485,-0.026499,-0.028466,-0.042018,-0.041425,0.009671,-0.013936,-0.012858,-0.012757,-0.015874,-0.024953,-0.021662,-0.028964,0.003282,0.020880,0.022164,0.026839,-0.088722,-0.089783,0.072530,0.046274,0.089259,0.086055,0.044832,0.201847,-0.021426,-0.014690,0.008369,0.146682,0.075066
P_emaildomain,-0.060268,-0.011988,0.012631,-0.060183,1.000000,0.597165,-0.004117,-0.004851,-0.003621,-0.025396,-0.015296,-0.016064,-0.015689,-0.014241,-0.014195,-0.014215,-0.014931,-0.017857,-0.015723,-0.014061,-0.015020,-0.014566,-0.006378,-0.008916,-0.007645,0.005153,-0.028220,-0.003336,0.011164,-0.009672,0.013575,-0.013486,-0.016270,-0.017712,-0.025558,-0.024706,-0.003239,-0.026691,-0.012727,-0.027462,-0.030116,-0.033564,-0.024801,-0.019891,-0.006691,0.007906,-0.006399,-0.006610,-0.039891,-0.028146,0.024277,0.017840,0.016214,0.018306,-0.003479,-0.057742,0.009923,0.052305,-0.008230,-0.007829,-0.007034
R_emaildomain,-0.048920,-0.004433,0.033378,-0.050792,0.597165,1.000000,0.002049,0.001214,0.003001,0.009833,-0.015374,-0.015983,-0.015630,-0.016002,-0.016653,-0.016333,-0.013767,-0.014956,-0.013831,-0.012748,-0.012718,-0.012669,-0.005398,-0.007537,-0.006473,0.009376,0.001867,0.002165,0.016490,-0.004098,-0.017819,-0.006576,-0.021258,-0.021360,0.000746,0.001171,-0.002798,-0.020614,-0.007685,-0.022297,-0.024518,-0.026467,-0.017466,-0.014202,0.019640,0.012198,0.021937,0.021464,0.010627,0.015665,0.012050,-0.001308,0.000942,0.002609,-0.001920,-0.037739,-0.003716,0.032778,-0.052601,-0.023050,-0.008005
C1,0.034436,-0.076772,-0.005254,0.013334,-0.004117,0.002049,1.000000,0.995925,

In [ ]:
#Lọc ra các cặp có giá trị corr > 0.5
corr_pairs = corr.unstack().sort_values(ascending=False)
corr_pairs = corr_pairs[corr_pairs < 1]

high_corr_pairs = corr_pairs[corr_pairs > 0.5]
high_corr_pairs = high_corr_pairs.iloc[::2]

In [ ]:
#Lọc ra các giá trị corr giữa các biến đầu vào và output
corr_label = corr['isFraud']

In [ ]:
#So sánh 2 biến có corr cao, nếu biến nào mà corr của nó với output thấp hơn thì sẽ được loại, ngược lại thì lưu
#Việc này giúp 2 điều: Xóa đi 1 giá trị trong cặp biến có Corr lớn, và xóa đi giá trị ít tác động đến đầu ra
columns_to_remove = []
for col1, col2 in high_corr_pairs.index:
  if col1 == 'isFraud' or col2 == 'isFraud':
    continue
  else:
    # So sánh giá trị trong corr_label và chọn cột có giá trị thấp hơn để xóa
    if corr_label[col1] < corr_label[col2]:
        columns_to_remove.append(col1)
    else:
        columns_to_remove.append(col2)

#Lọc đi các giá rị trùng lặp
columns_to_remove = list(set(columns_to_remove))
print(columns_to_remove)

['V233', 'V201', 'P_emaildomain', 'V243', 'V200', 'V111', 'C1', 'id_28', 'V284', 'V199', 'V109', 'V242', 'V121', 'V115', 'V113', 'V112', 'V186', 'V108', 'V286', 'id_15', 'ProductCD', 'V187', 'V116', 'V246', 'C11', 'V118', 'V288', 'V258', 'id_29', 'id_16', 'V244', 'V190', 'V289', 'V122', 'V119', 'V110']


In [ ]:
data = data.drop(columns = columns_to_remove)

***Đây là bộ dữ liệu sau cùng và dùng để training***

In [ ]:
data

,isFraud,card4,card6,R_emaildomain,C2,V98,V114,V117,V120,V173,...,V260,V297,id_12,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,0,1,0,0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0,21,0,0,0,1,1,539
1,0,0,1,0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0,1,0,0,1,1,1,1
2,0,1,0,0,4.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0,5,1,0,0,1,0,0
3,0,1,1,1,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0,5,1,0,0,1,0,0
4,0,0,1,0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0,5,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,0,1,1,28,9.0,0.0,1.0,1.0,1.0,0.0,...,2.0,0.0,0,16,1,0,0,0,1,333
144229,1,0,0,0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0,37,0,0,0,0,1,667
144230,0,1,0,0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0,13,1,0,0,0,1,719
144231,0,0,0,2,3.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0,10,0,0,0,0,0,2


In [ ]:
data.columns

Index(['isFraud', 'card4', 'card6', 'R_emaildomain', 'C2', 'V98', 'V114',
       'V117', 'V120', 'V173', 'V174', 'V194', 'V232', 'V240', 'V257', 'V260',
       'V297', 'id_12', 'id_31', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object')

# **PREDICTION**

In [ ]:
#Hàm đánh giá hiệu suất
def print_scores(y_test, y_pred):
  print(classification_report(y_test, y_pred))

#Hàm chia tập train-test và MinMax Scaler
def preprocess_data(data, test_size=0.2, random_state=42):

    # Chia tập train-test
    X = data.drop(columns=['isFraud'], axis=1)
    features = X.columns

    y = data['isFraud']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    #Standard Scaler
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess_data(data)

***1.Random Forest***

*Khá tốt*

In [ ]:
model_RF = RandomForestClassifier(n_estimators=200, random_state=42, class_weight = 'balanced')
model_RF.fit(X_train, y_train)

y_train_pred = model_RF.predict(X_train)
y_test_pred = model_RF.predict(X_test)

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       0.99      0.98      0.98    106328
           1       0.76      0.93      0.84      9058

    accuracy                           0.97    115386
   macro avg       0.88      0.95      0.91    115386
weighted avg       0.98      0.97      0.97    115386

              precision    recall  f1-score   support

           0       0.96      0.97      0.97     26587
           1       0.61      0.58      0.59      2260

    accuracy                           0.94     28847
   macro avg       0.79      0.77      0.78     28847
weighted avg       0.94      0.94      0.94     28847



***2.XGBOOST***

*Khả quan hơn Random Forest*

In [ ]:
model_XGB = xgb.XGBClassifier(n_estimators=200, random_state=42)
model_XGB.fit(X_train, y_train)

y_train_pred = model_XGB.predict(X_train)
y_test_pred = model_XGB.predict(X_test)

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       0.97      1.00      0.98    106328
           1       0.94      0.60      0.74      9058

    accuracy                           0.97    115386
   macro avg       0.95      0.80      0.86    115386
weighted avg       0.97      0.97      0.96    115386

              precision    recall  f1-score   support

           0       0.96      0.99      0.98     26587
           1       0.88      0.54      0.67      2260

    accuracy                           0.96     28847
   macro avg       0.92      0.77      0.82     28847
weighted avg       0.96      0.96      0.95     28847



***3.CATBOOST***

*Không ổn lắm*

In [ ]:
model_CB = CatBoostClassifier(iterations=200,random_seed=42)
model_CB.fit(X_train, y_train)

y_train_pred = model_CB.predict(X_train)
y_test_pred = model_CB.predict(X_test)

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

Learning rate set to 0.342317
0:	learn: 0.3561213	total: 85.6ms	remaining: 17s
1:	learn: 0.2511651	total: 120ms	remaining: 11.9s
2:	learn: 0.2194158	total: 152ms	remaining: 9.97s
3:	learn: 0.1995338	total: 205ms	remaining: 10s
4:	learn: 0.1934918	total: 236ms	remaining: 9.21s
5:	learn: 0.1884144	total: 272ms	remaining: 8.79s
6:	learn: 0.1821839	total: 308ms	remaining: 8.5s
7:	learn: 0.1784194	total: 344ms	remaining: 8.25s
8:	learn: 0.1769866	total: 376ms	remaining: 7.99s
9:	learn: 0.1757516	total: 415ms	remaining: 7.89s
10:	learn: 0.1740084	total: 451ms	remaining: 7.75s
11:	learn: 0.1732725	total: 482ms	remaining: 7.55s
12:	learn: 0.1724997	total: 513ms	remaining: 7.38s
13:	learn: 0.1717940	total: 551ms	remaining: 7.32s
14:	learn: 0.1704402	total: 584ms	remaining: 7.2s
15:	learn: 0.1696524	total: 619ms	remaining: 7.12s
16:	learn: 0.1690118	total: 652ms	remaining: 7.01s
17:	learn: 0.1681954	total: 682ms	remaining: 6.9s
18:	learn: 0.1673008	total: 728ms	remaining: 6.93s
19:	learn: 0.1668

***4.SMOTE kết hợp XGBOOST***

In [ ]:
# Áp dụng SMOTE để tăng cường lớp 1
class_counts = y_train.value_counts()
smote = SMOTE(sampling_strategy={1: int(0.75 * class_counts[0])}, random_state=42)
X_train_resample, y_train_resample = smote.fit_resample(X_train, y_train)

In [ ]:
model_XGB_2 = xgb.XGBClassifier(n_estimators=200, random_state=42)
model_XGB_2.fit(X_train_resample, y_train_resample)

y_train_pred = model_XGB_2.predict(X_train_resample)
y_test_pred = model_XGB_2.predict(X_test)

print_scores(y_train_resample, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       0.93      0.99      0.96    106328
           1       0.98      0.90      0.94     79746

    accuracy                           0.95    186074
   macro avg       0.96      0.95      0.95    186074
weighted avg       0.95      0.95      0.95    186074

              precision    recall  f1-score   support

           0       0.96      0.98      0.97     26587
           1       0.73      0.58      0.65      2260

    accuracy                           0.95     28847
   macro avg       0.85      0.78      0.81     28847
weighted avg       0.95      0.95      0.95     28847

